In [1]:
from cameo import load_model
from cameo import fba
from escher import Builder
import pandas as pd
import cobra

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
def simulate_model(experiment_id, model, objective, target, metabolite):
    model.objective = objective
    solution = fba(model)
    f_data = solution.fluxes
    f_data = f_data.rename(experiment_id)
    solution_result = []
    for i in target:
        flux = solution.fluxes.get(i)
        solution_result.append(flux)
    data = [experiment_id] + solution_result
    #print(data)
    df_target = pd.DataFrame(columns=[["Model"]+target], data=[data])
    print("experiment:", experiment_id, "| evaluate compound:", model.metabolites.get_by_id(metabolite).summary())
    return df_target, f_data

def run_series_simulation(experiment, model, mutant, target, metabolite):
    solution_fluxes = pd.DataFrame()
    df = pd.DataFrame()
    for i in experiment.index:
        experiment_id = experiment.loc[i, "experiment_id"]
        if experiment.loc[i, "model_id"] != "model":
            m = mutant
        else:
            m = model
        objective = experiment.loc[i, "objective"]
        data, sol = simulate_model(experiment_id, m, objective, target, metabolite)
        solution_fluxes = solution_fluxes.append(sol)
        df = df.append(data)
        print("****************************************", "\n")
    return df, solution_fluxes

In [3]:
model = load_model('iML1515')
model.solver = 'gurobi'
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f358718d3d0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [4]:
mutant = load_model('result/iML1515_xylitol_strain.xml')
mutant.solver = 'gurobi'
mutant

Name,iML1515
Memory address,0x07f3587dd2750
Number of metabolites,1879
Number of reactions,2715
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [5]:
experiment = pd.read_csv('data/experiment.txt')
experiment

,experiment_id,model_id,objective
0,wt_glu_biomass,model,BIOMASS_Ec_iML1515_core_75p37M
1,wt_glu_acetate,model,EX_ac_e
2,mt_glu_biomass,mutant,BIOMASS_Ec_iML1515_core_75p37M
3,mt_glu_acetate,mutant,EX_ac_e


In [6]:
target = ["BIOMASS_Ec_iML1515_core_75p37M", "ATPM", "ACODA", "CYSS", "UHGADA", "ACt4pp", "ACKr", "EX_ac_e", "XYLR"]
metabolite = "ac_c"

df, solution_fluxes = run_series_simulation(experiment, model, mutant, target, metabolite)

experiment: wt_glu_biomass | evaluate compound: ac_c
====
Formula: C2H3O2

Producing Reactions
-------------------
Percent     Flux Reaction                                 Definition
 50.36%   0.2594    ACODA           acorn_c + h2o_c --> ac_c + orn_c
  0.89% 0.004565   ACt4pp              ac_p + na1_p --> ac_c + na1_c
 42.13%    0.217     CYSS  acser_c + h2s_c --> ac_c + cys__L_c + h_c
  6.62%  0.03413   UHGADA         h2o_c + u3aga_c --> ac_c + u3hga_c

Consuming Reactions
-------------------
Percent      Flux Reaction                       Definition
 99.11%   -0.5106     ACKr  ac_c + atp_c <=> actp_c + adp_c
  0.89% -0.004565  ACt2rpp        ac_p + h_p <=> ac_c + h_c
**************************************** 

experiment: wt_glu_acetate | evaluate compound: ac_c
====
Formula: C2H3O2

Producing Reactions
-------------------
Percent Flux Reaction                       Definition
100.00% 28.8     ACKr  ac_c + atp_c <=> actp_c + adp_c

Consuming Reactions
-------------------
Percent  F

In [7]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_isobutyric_2.json',
    reaction_data=solution_fluxes.iloc[2],
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [8]:
df

,Model,BIOMASS_Ec_iML1515_core_75p37M,ATPM,ACODA,CYSS,UHGADA,ACt4pp,ACKr,EX_ac_e,XYLR
0,wt_glu_biomass,0.876997,6.86,0.259409,0.217041,0.034126,0.004565,0.510576,0.000000,None
0,wt_glu_acetate,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-28.795176,28.795176,None
0,mt_glu_biomass,0.777614,6.86,0.230012,0.192446,0.030259,0.000000,0.452716,0.000000,0
0,mt_glu_acetate,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-28.564311,28.564311,0


In [9]:
media_components = [["EX_xyl__D_e", 5],
                    ["EX_glc__D_e", 5]
                   ]

medium = model.medium
for i in media_components:
    medium[i[0]] = i[1]
    model.medium = medium
#print(model.medium)

medium = mutant.medium
for i in media_components:
    medium[i[0]] = i[1]
    mutant.medium = medium
#print(mutant.medium)

mutant.reactions.XYLK.knock_out()

target = ["BIOMASS_Ec_iML1515_core_75p37M", "ATPM", "ACODA", "CYSS", "UHGADA", "ACt4pp", "ACKr", "EX_ac_e", "XYLR"]
metabolite = "ac_c"
experiment2 = pd.read_csv('data/experiment2.txt')
df2, solution_fluxes2 = run_series_simulation(experiment2, model, mutant, target, metabolite)

experiment: wt_glu_xyl_biomass | evaluate compound: ac_c
====
Formula: C2H3O2

Producing Reactions
-------------------
Percent     Flux Reaction                                 Definition
 50.36%   0.2358    ACODA           acorn_c + h2o_c --> ac_c + orn_c
  0.89% 0.004149   ACt4pp              ac_p + na1_p --> ac_c + na1_c
 42.13%   0.1973     CYSS  acser_c + h2s_c --> ac_c + cys__L_c + h_c
  6.62%  0.03102   UHGADA         h2o_c + u3aga_c --> ac_c + u3hga_c

Consuming Reactions
-------------------
Percent      Flux Reaction                       Definition
 99.11%    -0.464     ACKr  ac_c + atp_c <=> actp_c + adp_c
  0.89% -0.004149  ACt2rpp        ac_p + h_p <=> ac_c + h_c
**************************************** 

experiment: wt_glu_xyl_acetate | evaluate compound: ac_c
====
Formula: C2H3O2

Producing Reactions
-------------------
Percent  Flux Reaction                       Definition
100.00% 26.28     ACKr  ac_c + atp_c <=> actp_c + adp_c

Consuming Reactions
-------------------


In [10]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution_fluxes2.iloc[4],
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'12DGR120tipp': 0.0, '12DGR140tipp': 0.0, '12DGR141tipp': 0.0, '12DGR160tipp': 0.0, '12…

In [11]:
df2

,Model,BIOMASS_Ec_iML1515_core_75p37M,ATPM,ACODA,CYSS,UHGADA,ACt4pp,ACKr,EX_ac_e,XYLR
0,wt_glu_xyl_biomass,0.797062,6.86,0.235764,0.197258,0.031015,0.004149,0.464038,0.000000,None
0,wt_glu_xyl_acetate,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-26.276735,26.276735,None
0,mt_glu_xyl_biomass,0.373402,6.86,0.110449,0.092410,0.014530,0.000000,0.217390,0.000000,0
0,mt_glu_xyl_acetate,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-13.953533,13.953533,0
0,mt_glu_xyl_acetate,0.000000,6.86,0.000000,0.000000,0.000000,0.000000,-3.953333,3.953333,5


In [12]:
reaction_1 = mutant.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M')
#reaction_1 = mutant.reactions.get_by_id('ATPM')
reaction_2 = mutant.reactions.get_by_id('EX_ac_e')
reaction_3 = mutant.reactions.get_by_id('EX_xylt_e')
new_objective = {reaction_1: 0.8, reaction_2:0.02, reaction_3:0.28}
mutant.objective = new_objective
#optimize the model with the new_objective
solution3 = mutant.optimize()
#print_solution(model)

In [13]:
builder = Builder(
    map_json='map/iJO1366.Central metabolism_rotation3.json',
    reaction_data=solution3.fluxes,
    #reaction_data=solution.fluxes,
)
builder

Builder(reaction_data={'CYTDK2': 0.0, 'XPPT': 0.0, 'HXPRT': 0.0, 'NDPK5': -0.004659101981087792, 'SHK3Dr': 0.0…

In [20]:
mutant.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0009268,0,0.00%
cl_e,EX_cl_e,0.0009268,0,0.00%
cobalt2_e,EX_cobalt2_e,4.451E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001262,0,0.00%
fe2_e,EX_fe2_e,0.00286,0,0.00%
glc__D_e,EX_glc__D_e,5,6,54.55%
k_e,EX_k_e,0.03476,0,0.00%
mg2_e,EX_mg2_e,0.001545,0,0.00%
mn2_e,EX_mn2_e,0.000123,0,0.00%
mobd_e,EX_mobd_e,1.246E-06,0,0.00%


In [15]:
cobra.io.write_sbml_model(mutant, "result/iML1515_xylitol_strain_tested.xml")